# Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# TODO: Import the necessary libs
# For example: 
import os

from lib.agents import Agent
# from lib.llm import LLM
# from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from dotenv import load_dotenv

In [3]:
 # TODO: Load environment variables
load_dotenv()

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

True

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

import chromadb
from lib.vector_db import VectorStoreManager

@tool
def retrieve_game(query: str) -> list[dict]:
    """Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry.

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    manager = VectorStoreManager(openai_api_key=os.getenv("OPENAI_API_KEY"))
    manager.chroma_client = chromadb.PersistentClient(path="chromadb")
    store = manager.get_store("udaplay")
    results = store.query(
        query_texts=[query],
        n_results=5
    )

    metadatas = results.get('metadatas')
    if metadatas:
        return metadatas[0]
    return []

#### Evaluate Retrieval Tool

In [5]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

from pydantic import BaseModel, Field
from lib.llm import LLM
import json

class EvaluationReport(BaseModel):
    """Data model for the evaluation report."""
    useful: bool = Field(description="Whether the documents are useful to answer the question.")
    description: str = Field(description="A detailed explanation of why the documents are useful or not.")

@tool
def evaluate_retrieval(question: str, retrieved_docs: list[dict]) -> dict:
    """Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.
    args:
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    judge = LLM(model="gpt-4o-mini")

    prompt = f"""Your task is to evaluate if the provided documents are sufficient to answer the user's query.
            Give a detailed explanation for your decision.

            User Query: {question}

            Retrieved Documents:
            {json.dumps(retrieved_docs, indent=2)}

            Based on the documents, can you confidently answer the user's query?
            """

    # Using the Pydantic model for structured output
    report = judge.invoke(
        input=prompt,
        response_format=EvaluationReport
    )

    return report.model_dump()

#### Game Web Search Tool

In [6]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry.

from tavily import TavilyClient

@tool
def game_web_search(question: str) -> list[dict]:
    """Searches the web for information about the video game industry.
    Use this when the internal database does not have the answer.
    args:
    - question: a question about game industry.
    """
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

    response = tavily_client.search(
        query=question,
        search_depth="advanced",
        max_results=5,
        include_answer=True
    )

    return response.get("results", [])

### Agent

In [7]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions
# Plug all Tools you developed

# System instructions for the agent
instructions = """You are UdaPlay, an AI assistant expert in the gaming industry.

Your goal is to answer the user's question in a clear and concise way, following these steps:
- First, try to answer the question using the `retrieve_game` tool.
- Second use the `evaluate_retrieval` tool to check if the retrieved information is enough to answer the question.
- If the information is not enough, use the `game_web_search` tool to search the web.
- Finally, combine all the information and provide a final answer to the user.
- Present the information in a clean and structured way.
- Always cite your sources.
"""

# Instantiate the Agent from the lib
agent = Agent(
    model_name="gpt-4o",
    instructions=instructions,
    tools=[
        retrieve_game,
        evaluate_retrieval,
        game_web_search
    ]
)

In [8]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X released for Playstation 5?

def run_agent(question: str):
    """Invokes the agent with a question and prints the final answer."""
    print(f"--- Running agent for: '{question}' ---\n")
    # The invoke method returns a Run object
    run_object = agent.invoke(question)
    final_state = run_object.get_final_state()

    # The final message is the last one in the list of messages
    final_message = final_state["messages"][-1]

    # Check the last message and print its content
    if final_message.role == "assistant":
        print("Final Answer:\n")
        print(final_message.content)
    else:
        print("Agent did not produce a final answer.")

    print(f"\n--- Run complete. Total tokens: {final_state.get('total_tokens', 0)} ---")

In [9]:
run_agent("When Pokémon Gold and Silver was released?")
run_agent("Which one was the first 3D platformer Mario game?")
run_agent("Was Mortal Kombat X realeased for Playstation 5?")

--- Running agent for: 'When Pokémon Gold and Silver was released?' ---

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[Tool Calling] Name: retrieve_game - Params: {'query': 'Pokémon Gold and Silver release date'}
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[Tool Calling] Name: evaluate_retrieval - Params: {'question': 'When Pokémon Gold and Silver was released?', 'retrieved_docs': ["{'Description': 'Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.', 'Name': 'Pokémon Gold and Silver', 'Publisher': 'Nintendo', 'Platform': 'Game Boy Color', 'YearOfRelease': 1999, 'Genre': 'Role-playing'}"]}
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Final Answer:

Pokémon Gold and Silver were released in 1999. These games are second-generation Pokémon g

### (Optional) Advanced

In [10]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes